# Data exploration

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import re
import pandas as pd

### Load code for this project

In [ ]:
import data_processing.generate_images as gi
%load_ext autoreload
%autoreload 1
%aimport data_processing.generate_images

### Investigate structure of data

In [ ]:
filename = 'data_processing/data/Mg22_alphaalpha_digiSim.h5'
hf = h5py.File(filename, "r")
hf.get('/get')

In [ ]:
event_i = 0
event = hf["Event_[{}]".format(event_i)][:]
display(pd.DataFrame(event))

### Length of events

In [ ]:
length = []
for key in hf.keys():
    length.append(len(hf[key]))
length = np.asarray(length)

In [ ]:
plt.figure()
plt.hist(length, bins=100)
plt.xlabel("Length (items)")
plt.show()

## Visualisation

In [ ]:
%matplotlib inline

event_i = 1

fig = plt.figure(figsize=(12,6))
ax = plt.subplot(131)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["y"], c=gi.get_event_by_index(hf, event_i)["A"], cmap='Greys')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("XY projection, linear scale")

ax = plt.subplot(132)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["y"], c=np.log(gi.get_event_by_index(hf, event_i)["A"]), cmap='Greys')
ax.set_xlabel("x")
ax.set_ylabel("z")
ax.set_title("XY projection, log scale")

#cbar = fig.colorbar(sc, orientation='vertical', label='A')    

plt.show()

In [ ]:
%matplotlib inline

event_i = 1

fig = plt.figure(figsize=(12,6))
ax = plt.subplot(131)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["y"], c=np.log(gi.get_event_by_index(hf, event_i)["A"]), cmap='Greys')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("XY projection")

ax = plt.subplot(132)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["z"], c=np.log(gi.get_event_by_index(hf, event_i)["A"]), cmap='Greys')
ax.set_xlabel("x")
ax.set_ylabel("z")
ax.set_title("XZ projection")

ax = plt.subplot(133)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["y"], gi.get_event_by_index(hf, event_i)["z"], c=np.log(gi.get_event_by_index(hf, event_i)["A"]), cmap='Greys')
ax.set_xlabel("y")
ax.set_ylabel("z")
ax.set_title("YZ projection")

cbar = fig.colorbar(sc, orientation='vertical', label='A')    

plt.show()

In [ ]:
n_rows, n_cols = 4, 4

fig = plt.figure(figsize=(12,12))

for i in range(n_rows*n_cols):
    ax = plt.subplot(n_rows, n_cols, i+1)
    sc = plt.scatter(gi.get_event_by_index(hf, i)["x"], gi.get_event_by_index(hf, i)["y"], c=np.log(gi.get_event_by_index(hf, i)["A"]), cmap='Greys')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Event: {}".format(i))
    #plt.xlim(-275.0, 275.0)
    #plt.ylim((-275.0, 275.0))
    #ax.set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.show()

## Read and label data - to dictionary

* Even events: beam -> label = 0
* Odd events: beam -> label = 1

In [ ]:
dict_data = gi.read_and_label_data("data_processing/data/")

## Discretization

From raw event to discretized image via transformation and normalization

In [ ]:
event_i = 1

In [ ]:
data = [list(dict_data[event_i])] #from dict to list (important to use 'list' as it copies the data!)

print("Shape:\n\tdata:", len(data))
data, max_charge = gi.transform_data(data)

features, targets = gi.make_image_features_targets(data, 'xy', 48)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1,2,1)
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["y"], c=np.log(gi.get_event_by_index(hf, event_i)["A"]), cmap='Greys', )
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Raw image")
plt.xlim(-275.0, 275.0)
plt.ylim((-275.0, 275.0))
cbar = fig.colorbar(sc, orientation='vertical', label='log(A)')
ax.set_aspect('equal', adjustable='box')

ax = fig.add_subplot(1,2,2)
im = ax.imshow(features[0], cmap='Greys')
ax.set_xlabel("Pixel along x")
ax.set_ylabel("Pixel along y")
ax.set_title("Discretized image")
plt.colorbar(im, ax=ax, label="RGB")

plt.tight_layout()
plt.show()

## Voxelized data

In [ ]:
import data_processing.generate_voxels as gv
%aimport data_processing.generate_voxels

In [ ]:
event_i = 9

xyzs = gv.discretize_grid_charge(gi.get_event_by_index(hf, event_i), 20, 20, 20)
voxels = xyzs.toarray()

mask = voxels[0] > 0

In [ ]:
bucket_num = np.array(range(len(voxels[0])))[mask]

x, y, z = gv.get_xyz_from_bucket(bucket_num, 20, 20, 20)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10,12))
ax = fig.add_subplot(211, projection='3d')
sc = plt.scatter(gi.get_event_by_index(hf, event_i)["x"], gi.get_event_by_index(hf, event_i)["y"], gi.get_event_by_index(hf, event_i)["z"], c=gi.get_event_by_index(hf, event_i)["A"], cmap='inferno', marker='.')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("Raw data")
plt.colorbar(sc, ax=ax, label='Charge')

ax = fig.add_subplot(212, projection='3d')
sc = plt.scatter(x, y, z, c=voxels[0][mask], cmap='inferno')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
#ax.set_xlim(0, gv.X_DISC)
#ax.set_ylim(0, gv.Y_DISC)
#ax.set_zlim(0, gv.Z_DISC)
ax.set_title("Discretized data")
plt.colorbar(sc, ax=ax, label='Scaled charge')

plt.show()